In [2]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress


def process_file(file_path):
    
    font_legend = {'family': 'TH Sarabun New',
                    'weight': 'bold',
                    'style': 'normal', 
                    'size': 18
                   }
    font_axis = {'family': 'TH Sarabun New',
                    'weight': 'bold',
                    'style': 'normal', 
                    'size': 20
                   }
    base_name   = os.path.splitext(os.path.basename(file_path))[0]
    parent_dir  = os.path.dirname(file_path)
    output_dir  = os.path.join(parent_dir, f"{base_name} plots")
    os.makedirs(output_dir, exist_ok=True)

    # Read CSV, skip first two summary rows, use third row as header
    df = pd.read_csv(file_path, delimiter=',', skiprows=2, header=0)

    # --- FULL PLOTS ---

    # 1) Mass Flow Rate vs Run Time (full) + linear fit
    x = df['Run Time [s]']
    y = df['Weight [Kg]']
    slope_full, intercept_full, r_full, _, _ = linregress(x, y)
    r2_full = r_full**2

    plt.figure()
    plt.plot(x, y, label='Data')
    plt.plot(x, slope_full*x + intercept_full,
             label=f'Fit: y={slope_full:.4f}x+{intercept_full:.4f}\nR^2={r2_full:.4f}')
    plt.xlabel('Run Time [s]', fontdict=font_axis)
    plt.ylabel('Weight [Kg]', fontdict=font_axis)
    plt.title('Mass Flow Rate vs Run Time', fontdict=font_axis)
    plt.legend(prop=font_legend)
    plt.ylim(bottom=0)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_full_plot.png"))
    plt.close()

    # 2) Differential Pressure vs Run Time (full)
    plt.figure()
    plt.plot(x, df['Differential Pressure [Bar]'])
    plt.xlabel('Run Time [s]', fontdict=font_axis)
    plt.ylabel('Differential Pressure [Bar]', fontdict=font_axis)
    plt.title('Differential Pressure vs Run Time', fontdict=font_axis)
    plt.ylim([0,3])
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Differential_Pressure_full_plot.png"))
    plt.close()

    # 3) Rotational Speed vs Run Time (full)
    plt.figure()
    plt.plot(x, df['Rotational Speed [RPM]'])
    plt.xlabel('Run Time [s]', fontdict=font_axis)
    plt.ylabel('Rotational Speed [RPM]', fontdict=font_axis)
    plt.title('Rotational Speed vs Run Time', fontdict=font_axis)
    plt.ylim([0,8500])
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Rotational_Speed_full_plot.png"))
    plt.close()

    # 4) Torque vs Run Time (full)
    plt.figure()
    plt.plot(x, df['Torque [N-m]'])
    plt.xlabel('Run Time [s]', fontdict=font_axis)
    plt.ylabel('Torque [N-m]', fontdict=font_axis)
    plt.title('Torque vs Run Time', fontdict=font_axis)
    plt.ylim([0,0.5])
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Torque_full_plot.png"))
    plt.close()

    # --- FILTERED PLOTS (Weight between 20 and 30 Kg) ---
    filtered_df = df[(df['Weight [Kg]'] >= 20) & (df['Weight [Kg]'] <= 30)]
    if not filtered_df.empty:
        frt = filtered_df['Run Time [s]'].iloc[-1] - filtered_df['Run Time [s]'].iloc[0]

        # 1) Mass Flow Rate (Filtered)
        xf = filtered_df['Run Time [s]']
        yf = filtered_df['Weight [Kg]']
        slope, intercept, r_val, _, _ = linregress(xf, yf)
        r2f = r_val**2

        plt.figure()
        plt.plot(xf, yf, label='Filtered Data')
        plt.plot(xf, slope*xf + intercept,
                 label=f'Fit: y={slope:.4f}x+{intercept:.4f}\nR^2={r2f:.4f}')
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Weight [Kg]', fontdict=font_axis)
        plt.title('Mass Flow Rate vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95, f'Filtered Run Time: {frt:.2f} s',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.legend(prop=font_legend)
        plt.ylim(bottom=0)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_filtered_plot.png"))
        plt.close()

        # 2) Differential Pressure (Filtered)
        dp = filtered_df['Differential Pressure [Bar]']
        avg_dp  = dp.mean()
        cnt_dp  = dp.count()
        std_dp  = dp.std(ddof=1)

        plt.figure()
        plt.plot(xf, dp)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Differential Pressure [Bar]', fontdict=font_axis)
        plt.title('Differential Pressure vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_dp:.4f} Bar\n'
                 f'Count: {cnt_dp}\n'
                 f'SD: {std_dp:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,3])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Differential_Pressure_filtered_plot.png"))
        plt.close()

        # 3) Rotational Speed (Filtered)
        rs = filtered_df['Rotational Speed [RPM]']
        avg_rs   = rs.mean()
        cnt_rs   = rs.count()
        std_rs   = rs.std(ddof=1)

        plt.figure()
        plt.plot(xf, rs)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Rotational Speed [RPM]', fontdict=font_axis)
        plt.title('Rotational Speed vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_rs:.4f} RPM\n'
                 f'Count: {cnt_rs}\n'
                 f'SD: {std_rs:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,8500])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Rotational_Speed_filtered_plot.png"))
        plt.close()

        # 4) Torque (Filtered)
        tq = filtered_df['Torque [N-m]']
        avg_tq  = tq.mean()
        cnt_tq  = tq.count()
        std_tq  = tq.std(ddof=1)

        plt.figure()
        plt.plot(xf, tq)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Torque [N-m]', fontdict=font_axis)
        plt.title('Torque vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_tq:.4f} N-m\n'
                 f'Count: {cnt_tq}\n'
                 f'SD: {std_tq:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,0.5])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Torque_filtered_plot.png"))
        plt.close()
    else:
        slope, intercept = np.nan, np.nan
        print(f"{base_name}: no data in weight range 20–30 Kg")

    # Return filtered fit parameters and stats for summary
    return {
        'file':             base_name,
        'Mass Flow Rate':   slope,
        'Intercept':        intercept,
        'avg_dp':           avg_dp if not filtered_df.empty else np.nan,
        'avg_rs':           avg_rs if not filtered_df.empty else np.nan,
        'avg_torque':       avg_tq if not filtered_df.empty else np.nan
    }


def main(csv_dir):
    files = glob.glob(os.path.join(csv_dir, '*.csv'))
    if not files:
        print(f"No CSV files found in {csv_dir}")
        return

    summaries = [process_file(f) for f in files]

    mfr = np.array([s['Mass Flow Rate'] for s in summaries])
    itc = np.array([s['Intercept']      for s in summaries])
    dp  = np.array([s['avg_dp']         for s in summaries])
    rs  = np.array([s['avg_rs']         for s in summaries])
    tq  = np.array([s['avg_torque']     for s in summaries])

    grand = {
        'mean_MFR':  np.nanmean(mfr),  'sd_MFR':  np.nanstd(mfr, ddof=1),
        'mean_int':  np.nanmean(itc),  'sd_int':  np.nanstd(itc, ddof=1),
        'mean_dp':   np.nanmean(dp),   'sd_dp':   np.nanstd(dp,  ddof=1),
        'mean_rs':   np.nanmean(rs),   'sd_rs':   np.nanstd(rs,  ddof=1),
        'mean_tq':   np.nanmean(tq),   'sd_tq':   np.nanstd(tq,  ddof=1)
    }

    print("\nPer-file filtered fit & stats:")
    for s in summaries:
        print(f" - {s['file']}: Mass Flow Rate={s['Mass Flow Rate']:.4f}, "
              f"Intercept={s['Intercept']:.4f}, ΔP={s['avg_dp']:.4f}, "
              f"RPM={s['avg_rs']:.2f}, Torque={s['avg_torque']:.4f}\n")

    print("Grand summary:\n")
    print(f" Mass Flow Rate: mean={grand['mean_MFR']:.4f}, SD={grand['sd_MFR']:.4f}")
    print(f" Intercept:      mean={grand['mean_int']:.4f}, SD={grand['sd_int']:.4f}")
    print(f" ΔP:             mean={grand['mean_dp']:.4f}, SD={grand['sd_dp']:.4f}")
    print(f" RPM:            mean={grand['mean_rs']:.2f}, SD={grand['sd_rs']:.2f}")
    print(f" Torque:         mean={grand['mean_tq']:.4f}, SD={grand['sd_tq']:.4f}\n")

    folder_name      = os.path.basename(csv_dir.rstrip(os.sep))
    summary_filename = f"{folder_name} summary.txt"
    summary_path     = os.path.join(csv_dir, summary_filename)

    with open(summary_path, 'w', encoding='utf-8') as f:
        f.write("Per-file filtered fit & stats:\n")
        for s in summaries:
            f.write(f"{s['file']}: Mass Flow Rate={s['Mass Flow Rate']:.4f}, "
                    f"ΔP={s['avg_dp']:.4f}, "
                    f"RPM={s['avg_rs']:.2f}, Torque={s['avg_torque']:.4f}\n")
        f.write("\nGrand summary:\n")
        f.write(f"\nMass Flow Rate: mean={grand['mean_MFR']:.4f}, SD={grand['sd_MFR']:.4f}\n")
        f.write(f"ΔP:             mean={grand['mean_dp']:.4f}, SD={grand['sd_dp']:.4f}\n")
        f.write(f"RPM:            mean={grand['mean_rs']:.2f}, SD={grand['sd_rs']:.2f}\n")
        f.write(f"Torque:         mean={grand['mean_tq']:.4f}, SD={grand['sd_tq']:.4f}")

    print(f"Summary written to {summary_path}")
    
    
if __name__ == '__main__':
    # 1) Base path (up to “Glycerine\”)
    BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'
    
    # 2) List the final two-level folders you want to process:
    subdirs = [
        r'24042025\Glycerine_8%_6730_0.081Kg-s',
        r'24042025\Glycerine_8%_7760_0.094Kg-s',
        r'23042025\Glycerine_8%_7250_0.094Kg-s',
        r'23042025\Glycerine_85%_6730_0.526Kg-s',
        r'23042025\Glycerine_85%_7760_0.603Kg-s',
        r'22042025\Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025\Glycerine_85%_7250_0.565Kg-s',
        r'21042025\Glycerine_43%_6730_0.416Kg-s',
        r'21042025\Glycerine_43%_7250_0.447Kg-s',
        r'21042025\Glycerine_43%_7760_0.477Kg-s',
        r'21042025\Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025\Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025\Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025\Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025\Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025\Glycerine_13%_6730_0.160Kg-s',
        r'19042025\Glycerine_13%_7760_0.188Kg-s',
        r'18042025\Glycerine_13%_7250_0.175Kg-s',
        r'18042025\Glycerine_19%_7760_0.268Kg-s',
        r'17042025\Glycerine_19%_6730_0.235Kg-s',
        r'17042025\Glycerine_19%_7250_0.253Kg-s',
        r'17042025\Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025\Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025\Glycerine_21%_6730_0.272Kg-s',
        r'26042025\Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025\Glycerine_21%_7760_0.313Kg-s',
        r'26042025\Glycerine_21%_7250_0.281Kg-s',
        r'28042025\Glycerine_13%_6730_0.160Kg-s',
        r'28042025\Glycerine_21%_7250_0.290Kg-s',
        r'28042025\Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025\Glycerine_36.5_7760_0.437Kg-s',
        r'28042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025\Glycerine_13%_7760_0.187Kg-s',
    ]

    # 3) Iterate through each and run
    for sub in subdirs:
        csv_directory = os.path.join(BASE_DIR, sub)
        print(f"\n=== Processing folder: {csv_directory} ===")
        main(csv_directory)



=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\24042025\Glycerine_8%_6730_0.081Kg-s ===

Per-file filtered fit & stats:
 - Glycerine_24042025_1354_6730RPM_8%_#1_0.081Kg-s: Mass Flow Rate=0.0797, Intercept=-1.9820, ΔP=1.9931, RPM=6729.90, Torque=0.3364

 - Glycerine_24042025_1412_6730RPM_8%_#2_0.081Kg-s: Mass Flow Rate=0.0802, Intercept=0.2094, ΔP=1.9987, RPM=6730.00, Torque=0.3304

 - Glycerine_24042025_1430_6730RPM_8%_#3_0.080Kg-s: Mass Flow Rate=0.0798, Intercept=0.1649, ΔP=1.9987, RPM=6730.06, Torque=0.3272

 - Glycerine_24042025_1448_6730RPM_8%_#4_0.080Kg-s: Mass Flow Rate=0.0796, Intercept=0.1040, ΔP=2.0049, RPM=6729.99, Torque=0.3152

 - Glycerine_24042025_1506_6730RPM_8%_#5_0.079Kg-s: Mass Flow Rate=0.0787, Intercept=-2.1400, ΔP=2.0072, RPM=6729.99, Torque=0.2865

Grand summary:

 Mass Flow Rate: mean=0.0796, SD=0.0005
 Intercept:      mean=-0.7288, SD=1.2180
 ΔP:             mean=2.0005, SD=0.

In [23]:
 subdirs = [
        r'24042025/Glycerine_8%_6730_0.081Kg-s',
        r'24042025/Glycerine_8%_7760_0.094Kg-s',
        r'23042025/Glycerine_8%_7250_0.094Kg-s',
        r'23042025/Glycerine_85%_6730_0.526Kg-s',
        r'23042025/Glycerine_85%_7760_0.603Kg-s',
        r'22042025/Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025/Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025/Glycerine_85%_7250_0.565Kg-s',
        r'21042025/Glycerine_43%_6730_0.416Kg-s',
        r'21042025/Glycerine_43%_7250_0.447Kg-s',
        r'21042025/Glycerine_43%_7760_0.477Kg-s',
        r'21042025/Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025/Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025/Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025/Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025/Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025/Glycerine_13%_6730_0.160Kg-s',
        r'19042025/Glycerine_13%_7760_0.188Kg-s',
        r'18042025/Glycerine_13%_7250_0.175Kg-s',
        r'18042025/Glycerine_19%_7760_0.268Kg-s',
        r'17042025/Glycerine_19%_6730_0.235Kg-s',
        r'17042025/Glycerine_19%_7250_0.253Kg-s',
        r'17042025/Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025/Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025/Glycerine_21%_6730_0.272Kg-s',
        r'26042025/Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025/Glycerine_21%_7760_0.313Kg-s',
        r'26042025/Glycerine_21%_7250_0.281Kg-s',
        r'28042025/Glycerine_13%_6730_0.160Kg-s',
        r'28042025/Glycerine_21%_7250_0.290Kg-s',
        r'28042025/Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025/Glycerine_36.5_7760_0.437Kg-s',
        r'28042025/Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025/Glycerine_13%_7760_0.187Kg-s',
    ]
 
   BASE_DIR = r'/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine'
 
 
 
 #Windows
 
  subdirs = [
        r'24042025\Glycerine_8%_6730_0.081Kg-s',
        r'24042025\Glycerine_8%_7760_0.094Kg-s',
        r'23042025\Glycerine_8%_7250_0.094Kg-s',
        r'23042025\Glycerine_85%_6730_0.526Kg-s',
        r'23042025\Glycerine_85%_7760_0.603Kg-s',
        r'22042025\Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025\Glycerine_85%_7250_0.565Kg-s',
        r'21042025\Glycerine_43%_6730_0.416Kg-s',
        r'21042025\Glycerine_43%_7250_0.447Kg-s',
        r'21042025\Glycerine_43%_7760_0.477Kg-s',
        r'21042025\Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025\Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025\Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025\Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025\Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025\Glycerine_13%_6730_0.160Kg-s',
        r'19042025\Glycerine_13%_7760_0.188Kg-s',
        r'18042025\Glycerine_13%_7250_0.175Kg-s',
        r'18042025\Glycerine_19%_7760_0.268Kg-s',
        r'17042025\Glycerine_19%_6730_0.235Kg-s',
        r'17042025\Glycerine_19%_7250_0.253Kg-s',
        r'17042025\Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025\Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025\Glycerine_21%_6730_0.272Kg-s',
        r'26042025\Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025\Glycerine_21%_7760_0.313Kg-s',
        r'26042025\Glycerine_21%_7250_0.281Kg-s',
        r'28042025\Glycerine_13%_6730_0.160Kg-s',
        r'28042025\Glycerine_21%_7250_0.290Kg-s',
        r'28042025\Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025\Glycerine_36.5_7760_0.437Kg-s',
        r'28042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025\Glycerine_13%_7760_0.187Kg-s',
    ]

  BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'

IndentationError: unindent does not match any outer indentation level (<string>, line 33)

In [3]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.font_manager as font_manager

def process_file(file_path):
    font_legend = {'family': 'TH Sarabun New',
                    'weight': 'bold',
                    'style': 'normal', 
                    'size': 18
                   }
    font_axis = {'family': 'TH Sarabun New',
                    'weight': 'bold',
                    'style': 'normal', 
                    'size': 20
                   }
    base_name   = os.path.splitext(os.path.basename(file_path))[0]
    parent_dir  = os.path.dirname(file_path)
    output_dir  = os.path.join(parent_dir, f"{base_name} plots")
    output_dir  = os.path.join(output_dir, f"combined plots")
    os.makedirs(output_dir, exist_ok=True)

    # Read CSV, skip first two summary rows, use third row as header
    df = pd.read_csv(file_path, delimiter=',', skiprows=2, header=0)

    # --- COMBINED FULL PLOTS ---

    # 1) Torque and Pressure vs Weight
    x = df['Weight [Kg]']
    #y = df['Weight [Kg]']

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    #plt.plot(x, y, label='Data')
    #plt.xlabel('Run Time [s]')
    #plt.ylabel('Weight [Kg]')
    #plt.title('Mass Flow Rate vs Run Time')
    #plt.legend()
    #plt.ylim(bottom=0)
    #plt.tight_layout()

    lns1 = ax1.plot(x, df['Differential Pressure [Bar]'],label='Differential Pressure')
    ax1.set_xlabel('Weight [Kg]', fontdict=font_axis)
    ax1.set_ylabel('Differential Pressure [Bar]', fontdict=font_axis)
    ax1.set_ylim(bottom=0)

    lns2 = ax2.plot(x, df["Torque [N-m]"],color="orange",label='Torque')
    ax2.set_ylabel('Torque [N-m]', fontdict=font_axis)
    ax2.set_ylim([0,0.6])
    
    lns = lns1 + lns2
    
    labs = [l.get_label() for l in lns]       
    ax1.legend(lns, labs, loc="lower right",prop=font_legend)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Pressure_&_Torque_vs_Weight_full_combined_plot.png"))
    plt.close()
    
    # 2) Torque and Pressure vs Time
    x = df['Run Time [s]']
    #y = df['Weight [Kg]']

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    #plt.plot(x, y, label='Data')
    #plt.xlabel('Run Time [s]')
    #plt.ylabel('Weight [Kg]')
    #plt.title('Mass Flow Rate vs Run Time')
    #plt.legend()
    #plt.ylim(bottom=0)
    #plt.tight_layout()

    lns1 = ax1.plot(x, df['Differential Pressure [Bar]'],label='Differential Pressure')
    ax1.set_xlabel('Run Time [s]', fontdict=font_axis)
    ax1.set_ylabel('Differential Pressure [Bar]', fontdict=font_axis)
    ax1.set_ylim(bottom=0)
    
    lns2 = ax2.plot(x, df["Torque [N-m]"],color="orange",label='Torque')
    ax2.set_ylabel('Torque [N-m]',fontsize = 16, fontdict=font_axis)
    ax2.set_ylim([0,0.6])
    
    lns = lns1 + lns2

    labs = [l.get_label() for l in lns]       
    ax1.legend(lns, labs, loc="lower right",prop=font_legend)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Pressure_&_Torque_vs_Run_Time_full_combined_plot.png"))
    plt.close()

def main(csv_dir):
    files = glob.glob(os.path.join(csv_dir, '*.csv'))
    if not files:
        print(f"No CSV files found in {csv_dir}")
        return

    summaries = [process_file(f) for f in files]

    
    
if __name__ == '__main__':
    # 1) Base path (up to “Glycerine\”)
    BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'
    
    # 2) List the final two-level folders you want to process:
    subdirs = [
        r'24042025\Glycerine_8%_6730_0.081Kg-s',
        r'24042025\Glycerine_8%_7760_0.094Kg-s',
        r'23042025\Glycerine_8%_7250_0.094Kg-s',
        r'23042025\Glycerine_85%_6730_0.526Kg-s',
        r'23042025\Glycerine_85%_7760_0.603Kg-s',
        r'22042025\Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025\Glycerine_85%_7250_0.565Kg-s',
        r'21042025\Glycerine_43%_6730_0.416Kg-s',
        r'21042025\Glycerine_43%_7250_0.447Kg-s',
        r'21042025\Glycerine_43%_7760_0.477Kg-s',
        r'21042025\Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025\Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025\Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025\Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025\Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025\Glycerine_13%_6730_0.160Kg-s',
        r'19042025\Glycerine_13%_7760_0.188Kg-s',
        r'18042025\Glycerine_13%_7250_0.175Kg-s',
        r'18042025\Glycerine_19%_7760_0.268Kg-s',
        r'17042025\Glycerine_19%_6730_0.235Kg-s',
        r'17042025\Glycerine_19%_7250_0.253Kg-s',
        r'17042025\Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025\Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025\Glycerine_21%_6730_0.272Kg-s',
        r'26042025\Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025\Glycerine_21%_7760_0.313Kg-s',
        r'26042025\Glycerine_21%_7250_0.281Kg-s',
        r'28042025\Glycerine_13%_6730_0.160Kg-s',
        r'28042025\Glycerine_21%_7250_0.290Kg-s',
        r'28042025\Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025\Glycerine_36.5_7760_0.437Kg-s',
        r'28042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025\Glycerine_13%_7760_0.187Kg-s',
    ]

    # 3) Iterate through each and run
    for sub in subdirs:
        csv_directory = os.path.join(BASE_DIR, sub)
        print(f"\n=== Processing folder: {csv_directory} ===")
        main(csv_directory)



=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\24042025\Glycerine_8%_6730_0.081Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\24042025\Glycerine_8%_7760_0.094Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\23042025\Glycerine_8%_7250_0.094Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\23042025\Glycerine_85%_6730_0.526Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\23042025\Glycerine_85%_7760_0.603Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glyce